In [1]:
import pandas as pd
from numpy import median
from datetime import date

In [2]:
# Hier euren Pfad zur CSV einfügen
path = "C:/Users/lorga/Desktop/Parlament/data/"
meps_former_df = pd.read_csv(path + "output_former.csv", sep = ";")
meps_current_df = pd.read_csv(path + "output.csv", sep = ";")

In [3]:
meps_former_df.head()

,identifier,name,familyName,givenName,country,group,gender,memberships,born_place,born_day,born_month,born_year,disability,relatives,educated_at,highest_degree,occupation
0,197483,sylvia limmer,Limmer,Sylvia,DE,NI,FEMALE,NaN,Bayreuth,8,2,1966,NaN,NaN,NaN,NaN,"politician,doctor"
1,99945,lena düpont,Düpont,Lena,DE,PPE,FEMALE,"LIBE,D-ME,AGRI,FEMM,D-JP",Dortmund,30,4,1986,NaN,NaN,NaN,university,"politician,media,manager"
2,204333,salvatore de meo,De Meo,Salvatore,IT,PPE,MALE,"AFCO,AGRI,D-RS,D-IN,ITRE,IMCO,D-RU",Fondi,27,10,1971,NaN,NaN,Sapienza University of Rome,university,"politician,lawyer,businessperson,manager"
3,124739,carlos zorrinho,Zorrinho,Carlos,PT,S&D,MALE,"DACP,ITRE,DAFR,DEVE,DMAS",Óbidos,28,5,1959,NaN,NaN,University of Évora,phd,"official,politician,researcher"
4,97236,marie toussaint,Toussaint,Marie,FR,Verts/ALE,FEMALE,"ITRE,JURI,D-US,DLAT,ENVI",LILLE,27,5,1987,NaN,NaN,Sciences Po,NaN,"politician,lawyer,activist"


In [4]:
meps_current_df.head()

,identifier,name,familyName,givenName,country,group,gender,memberships,born_place,born_day,born_month,born_year,disability,relatives,educated_at,highest_degree,occupation
0,197483,sylvia limmer,Limmer,Sylvia,DE,NI,FEMALE,NaN,Bayreuth,8,2,1966,NaN,NaN,NaN,NaN,"politician,doctor"
1,99945,lena düpont,Düpont,Lena,DE,PPE,FEMALE,"LIBE,D-ME,AGRI,FEMM,D-JP",Dortmund,30,4,1986,NaN,NaN,NaN,university,"politician,media,manager"
2,204333,salvatore de meo,De Meo,Salvatore,IT,PPE,MALE,"AFCO,AGRI,D-RS,D-IN,ITRE,IMCO,D-RU",Fondi,27,10,1971,NaN,NaN,Sapienza University of Rome,university,"politician,lawyer,businessperson,manager"
3,124739,carlos zorrinho,Zorrinho,Carlos,PT,S&D,MALE,"DACP,ITRE,DAFR,DEVE,DMAS",Óbidos,28,5,1959,NaN,NaN,University of Évora,phd,"official,politician,researcher"
4,97236,marie toussaint,Toussaint,Marie,FR,Verts/ALE,FEMALE,"ITRE,JURI,D-US,DLAT,ENVI",LILLE,27,5,1987,NaN,NaN,Sciences Po,NaN,"politician,lawyer,activist"


In [5]:
group_list = list(set(meps_current_df["group"].tolist()))
country_list = list(set(meps_current_df["country"].tolist()))
org_list = []
for list_entry in meps_current_df["memberships"].tolist():
    if list_entry:
        for entry in str(list_entry).split(","):
            org_list.append(entry)
org_list = list(set(org_list))
org_list.remove("nan")
committee_list = [org for org in org_list if org[0] != "D"]

In [21]:
def filter_returned(column, column_entries, current_df, former_df):
    return_dict = {}
    for column_entry in column_entries:
        return_counter = 0
        filter_current_df = current_df.loc[current_df[column].notna()]
        filter_current_df = filter_current_df.loc[filter_current_df[column].str.contains(column_entry)]
        filter_former_df = former_df.loc[former_df[column].notna()]
        filter_former_df = filter_former_df.loc[filter_former_df[column].str.contains(column_entry)]
        current_identifiers = filter_current_df["identifier"].tolist()
        former_identifiers = filter_former_df["identifier"].tolist()
        for identifier in current_identifiers:
            if identifier in former_identifiers:
                return_counter += 1
        return_ratio = "????"
        # Variante 1: Wie viele der aktuellen MEPs sind Rückkehrer?
        returned_current_ratio = return_counter / len(current_identifiers)
        # Variante 2: Wie viele der ehemaligen MEPs sind zurückgekehrt?
        returned_former_ratio = return_counter / len(former_identifiers)
        return_dict[column_entry] = [return_counter, len(current_identifiers), len(former_identifiers), 
                                     returned_current_ratio, returned_former_ratio]
    return_df = pd.DataFrame.from_dict(return_dict, orient = "index")#.transpose()
    return_df = return_df.reset_index()
    return_df = return_df.rename(columns = {"index": column, 0: "returned_meps", 1: "current_meps", 2: "former_meps", 
                                            3: "current_return_ratio", 4: "former_return_ratio"})
    return return_df.sort_values("current_return_ratio")

In [22]:
group_returned_df = filter_returned("group", group_list, meps_current_df, meps_former_df)
group_returned_df.to_csv("followup_group_returned.csv", index = False)
group_returned_df

,group,returned_meps,current_meps,former_meps,current_return_ratio,former_return_ratio
0,ID,49,49,49,1.0,1.0
1,S&D,139,139,139,1.0,1.0
2,The Left,37,37,37,1.0,1.0
3,NI,62,62,62,1.0,1.0
4,ECR,69,69,69,1.0,1.0
5,Renew,102,102,102,1.0,1.0
6,PPE,176,176,176,1.0,1.0
7,Verts/ALE,71,71,71,1.0,1.0


In [23]:
country_returned_df = filter_returned("country", country_list, meps_current_df, meps_former_df)
country_returned_df.to_csv("followup_country_returned.csv", index = False)
country_returned_df

,country,returned_meps,current_meps,former_meps,current_return_ratio,former_return_ratio
0,ES,59,59,59,1.0,1.0
24,GR,21,21,21,1.0,1.0
23,LT,11,11,11,1.0,1.0
22,BE,21,21,21,1.0,1.0
21,CZ,21,21,21,1.0,1.0
20,LU,6,6,6,1.0,1.0
19,RO,33,33,33,1.0,1.0
18,PL,52,52,52,1.0,1.0
17,FR,79,79,79,1.0,1.0
16,DE,96,96,96,1.0,1.0


In [25]:
committee_returned_df = filter_returned("memberships", committee_list, meps_current_df, meps_former_df)
committee_returned_df.to_csv("followup_committee_returned.csv", index = False)
committee_returned_df = committee_returned_df.rename(columns = {"memberships": "committee"})
committee_returned_df

,committee,returned_meps,current_meps,former_meps,current_return_ratio,former_return_ratio
0,INTA,81,81,81,1.0,1.0
19,JURI,47,47,47,1.0,1.0
18,PETI,65,65,65,1.0,1.0
17,PECH,50,50,50,1.0,1.0
16,SANT,54,54,54,1.0,1.0
15,LIBE,132,132,132,1.0,1.0
14,FEMM,64,64,64,1.0,1.0
13,CULT,57,57,57,1.0,1.0
12,AFCO,54,54,54,1.0,1.0
11,AFET,151,151,151,1.0,1.0


In [10]:
current_gender_df = pd.DataFrame(meps_current_df.value_counts("gender"))
current_gender_df = current_gender_df.reset_index().rename(columns = {0: "current_count"})
current_gender_df["current_pc"] = current_gender_df["current_count"] / current_gender_df["current_count"].sum()
former_gender_df = pd.DataFrame(meps_former_df.value_counts("gender"))
former_gender_df = former_gender_df.reset_index().rename(columns = {0: "former_count"})
former_gender_df["former_pc"] = former_gender_df["former_count"] / former_gender_df["former_count"].sum()
gender_df = pd.merge(current_gender_df, former_gender_df, on = "gender", how = "outer")
gender_df["change_count"] = gender_df["former_count"] - gender_df["current_count"]
gender_df["change_pc"] = gender_df["former_pc"] - gender_df["current_pc"]
gender_df.to_csv("followup_gender.csv", index = False)
gender_df

,gender,current_count,current_pc,former_count,former_pc,change_count,change_pc
0,MALE,425,0.602837,425,0.602837,0,0.0
1,FEMALE,280,0.397163,280,0.397163,0,0.0


In [11]:
def filter_men_perc(column, column_entries, df):
    gender_dict = {}
    non_na_df = df.loc[df[column].notna()]
    for column_entry in column_entries:
        filter_df = non_na_df.loc[non_na_df[column].str.contains(column_entry)]
        female_count = len(filter_df.loc[filter_df["gender"] == "MALE"].index)
        if len(filter_df.index > 0):
            female_percentage = female_count / len(filter_df.index) * 100
            gender_dict[column_entry] = female_percentage
    filter_percentage_df = pd.DataFrame.from_dict([gender_dict]).transpose()
    filter_percentage_df = filter_percentage_df.reset_index()
    filter_percentage_df = filter_percentage_df.rename(columns = {"index": column, 0: "men_percentage"})
    return filter_percentage_df.sort_values("men_percentage")

In [12]:
def compare_current_former(function, column, column_entries, meps_current_df, meps_former_df):
    current_df = function(column, column_entries, meps_current_df)
    former_df = function(column, column_entries, meps_former_df)
    compare_df = pd.merge(current_df, former_df, 
                            on = column, how = "outer", suffixes = ("_current", "_former"))
    compare_df["change"] = compare_df.iloc[:, -2] - compare_df.iloc[:, -1]
    return compare_df

In [13]:
group_men_df = compare_current_former(filter_men_perc, "group", group_list, meps_current_df, meps_former_df)
group_men_df.to_csv("followup_group_men.csv", index = False)
group_men_df

,group,men_percentage_current,men_percentage_former,change
0,The Left,48.648649,48.648649,0.0
1,Verts/ALE,52.112676,52.112676,0.0
2,S&D,57.553957,57.553957,0.0
3,Renew,57.843137,57.843137,0.0
4,ID,61.224490,61.224490,0.0
5,PPE,62.500000,62.500000,0.0
6,NI,66.129032,66.129032,0.0
7,ECR,72.463768,72.463768,0.0


In [14]:
country_men_df = compare_current_former(filter_men_perc, "country", country_list, meps_current_df, meps_former_df)
country_men_df.to_csv("followup_country_men.csv", index = False)
country_men_df

,country,men_percentage_current,men_percentage_former,change
0,LU,33.333333,33.333333,0.0
1,FI,35.714286,35.714286,0.0
2,SE,47.619048,47.619048,0.0
3,ES,49.152542,49.152542,0.0
4,FR,51.898734,51.898734,0.0
5,PT,52.380952,52.380952,0.0
6,IT,53.947368,53.947368,0.0
7,NL,55.172414,55.172414,0.0
8,DK,57.142857,57.142857,0.0
9,IE,61.538462,61.538462,0.0


In [15]:
committee_men_df = compare_current_former(filter_men_perc, "memberships", committee_list, meps_current_df, meps_former_df)
committee_men_df = committee_men_df.rename(columns = {"memberships": "committee"})
committee_men_df.to_csv("followup_committee_men.csv", index = False)
committee_men_df

,committee,men_percentage_current,men_percentage_former,change
0,FEMM,14.062500,14.062500,0.0
1,EMPL,43.564356,43.564356,0.0
2,SANT,48.148148,48.148148,0.0
3,ENVI,52.976190,52.976190,0.0
4,PECH,56.000000,56.000000,0.0
5,IMCO,56.097561,56.097561,0.0
6,LIBE,56.818182,56.818182,0.0
7,REGI,57.831325,57.831325,0.0
8,CULT,57.894737,57.894737,0.0
9,AGRI,60.674157,60.674157,0.0


In [16]:
def calculate_age(born_year, born_month, born_day):
    #today = date.today()
    today = date(2024, 6, 16) # Doublecheck
    if born_month < today.month:
        return today.year - born_year
    elif born_month == today.month:
        if born_day <= today.day:
            return today.year - born_year
        return today.year - born_year - 1
    return today.year - born_year - 1   

In [17]:
def get_median_age(column, filter_list, meps_df):
    df = meps_df.loc[meps_df[column].notna()]
    df = df.loc[meps_df["born_year"] != 0]
    df = df.loc[meps_df["born_year"].notna()]
    median_age_dict = {}
    for filter_entry in filter_list:
        filter_df = df.loc[df[column].str.contains(filter_entry)]
        ages = filter_df.apply(lambda x: calculate_age(x["born_year"], x["born_month"], x["born_day"]), axis = 1)
        ages = sorted(ages)
        median_age_dict[filter_entry] = median(ages)
    median_age_df = pd.DataFrame.from_dict(median_age_dict, orient = "index")
    median_age_df = median_age_df.reset_index().rename(columns = {"index": column, 0: "median_age"})
    median_age_df = median_age_df.sort_values("median_age")
    return median_age_df

In [18]:
group_age_df = compare_current_former(get_median_age, "group", group_list, meps_current_df, meps_former_df)
group_age_df.to_csv("followup_group_age.csv", index = False)
group_age_df

,group,median_age_current,median_age_former,change
0,Verts/ALE,48.0,48.0,0.0
1,The Left,51.0,51.0,0.0
2,Renew,52.0,52.0,0.0
3,NI,54.0,54.0,0.0
4,S&D,55.0,55.0,0.0
5,PPE,55.0,55.0,0.0
6,ID,57.0,57.0,0.0
7,ECR,58.0,58.0,0.0


In [19]:
country_age_df = compare_current_former(get_median_age, "country", country_list, meps_current_df, meps_former_df)
country_age_df.to_csv("followup_country_age.csv", index = False)
country_age_df

,country,median_age_current,median_age_former,change
0,MT,43.5,43.5,0.0
1,PT,47.0,47.0,0.0
2,CZ,48.0,48.0,0.0
3,HU,49.0,49.0,0.0
4,NL,50.0,50.0,0.0
5,SE,50.0,50.0,0.0
6,SK,51.0,51.0,0.0
7,RO,53.0,53.0,0.0
8,ES,53.0,53.0,0.0
9,BG,53.0,53.0,0.0


In [26]:
committee_age_list = compare_current_former(get_median_age, "memberships", committee_list, meps_current_df, meps_former_df)
committee_men_df = committee_men_df.rename(columns = {"memberships": "committee"})
committee_men_df.to_csv("followup_committee_age.csv", index = False)
committee_age_list

,memberships,median_age_current,median_age_former,change
0,LIBE,49.0,49.0,0.0
1,SANT,50.0,50.0,0.0
2,IMCO,50.5,50.5,0.0
3,PETI,51.0,51.0,0.0
4,FEMM,52.0,52.0,0.0
5,JURI,52.0,52.0,0.0
6,ITRE,53.0,53.0,0.0
7,TRAN,53.0,53.0,0.0
8,EMPL,53.0,53.0,0.0
9,REGI,53.0,53.0,0.0
